# DI Pre Processing
Corrects png files and store them in a new folder
<br>
<br>
Usage: 
1) Run notebook
2) Enter drive where video files are stored 
<br>
<br>

Notes:
1) Needs JSON files that contains metadata
2) Update root directory
<br>
<br>
![UofC logo](./pictures/uofc_logo-black.jpg)

In [1]:
#import libraries
import os
import cv2
import json
import pandas as pd
import numpy as np
from timeit import default_timer as timer
import psutil
import traceback

In [2]:
# make sure to update path
user_drive = input("Enter user drive: ").upper()
video_path = f"{user_drive}:/Christian/DI_centre_structured"
input(f"Is this the right directory - {video_path}?")

''

In [3]:
VIDEO_CHARACTERISTICS = {
    "With Blankets" : "WB",
    "B" : "WB",
    "Without Blankets" : "WOB",
    "WOB": "WOB",
    "3 Meters" : "3m",
    "2 Meters" : "2m",
    "Hold Breath" : "HB",
    "Hold Breathe" : "HB",
    "H" : "HB",
    "Relaxed" : "rel",
    "R": "rel",
}

In [4]:
repo_dir = os.getcwd()
json_dir = repo_dir + "/records/JSON"
log_dir = repo_dir + "/records/logs"

In [5]:
""" creates unique id for each scenario """

def get_id(patient_data: dict, video_count:int) -> str:
    alias = patient_data["alias"]
    blanket = VIDEO_CHARACTERISTICS.get(patient_data["blanket"], "?")
    distance = VIDEO_CHARACTERISTICS.get(patient_data["distance"].title(), "?")
    breathing = VIDEO_CHARACTERISTICS.get(patient_data["breathing"], "?")
    id = alias + "-" + distance + "-" + blanket + "-" + breathing
    id = f"{alias}_{video_count}-{distance}-{blanket}-{breathing}"
    return id

In [6]:
""" creates folder for exported frames if it does not exist """

def set_folder(save_folder:str) -> None:
    if not os.path.isdir(save_folder):
        os.mkdir(save_folder)
    else:
        for files in os.listdir(save_folder):
            os.remove(os.path.join(save_folder, files))

In [7]:
class NotEnoughFreeSpace(Exception):
    pass

In [8]:
""" gets drives storage """

def get_drive_storage(user_drive:str):
    try:
        drive_usage = psutil.disk_usage(user_drive + ":\\")
        
        # Convert bytes to GB
        total_space = drive_usage.total / (1024**3)
        used_space = drive_usage.used / (1024**3)
        free_space = drive_usage.free / (1024**3)
        
        return {
            'drive_letter': user_drive,
            'total_space_GB': total_space,
            'used_space_GB': used_space,
            'free_space_GB': free_space,
        }
    except FileNotFoundError:
        return f"Drive {user_drive} not found."

def check_drive_usage(user_drive:str):
    minimum_space = 75
    drive_info = get_drive_storage(user_drive)

    if drive_info["free_space_GB"] < minimum_space:
        raise NotEnoughFreeSpace(f"Not enough free space on Drive {user_drive}. Minimum {minimum_space} GB required.")    

In [9]:
""" re samples number of frames """

def up_sample(old_fps: int, new_fps: int, start:int) -> list[int]:
    frames_arr = np.arange(start, old_fps, dtype=int)
    frames_interp = np.linspace(start, old_fps-1, new_fps)
    nearest_indices = np.round(frames_interp).astype(int)
    up_sampled_list = np.take(frames_arr, nearest_indices, mode='wrap')

    return up_sampled_list.tolist()

In [10]:
""" checks each png for corruption """

def is_png_corrupted(file_path):
    try:
        image = cv2.imread(file_path, cv2.IMREAD_UNCHANGED)
        if image is None:
            return True
        return False
    except Exception:
        return True

def check_for_corrupted_png_files(folder_path):
    corrupted_files = []
    for root, _, files in os.walk(folder_path):
        for file in files:
            if file.lower().endswith(".png"):
                file_path = os.path.join(root, file)
                if is_png_corrupted(file_path):
                    corrupted_files.append(file_path)
    
    if len(corrupted_files) != 0:
        print("Corrupted PNG files found:")
        for file_path in corrupted_files:
            print(file_path)           
    
    return corrupted_files

In [11]:
""" reads JSON file """
def load_json(json_dir:str, filename:str) -> dict:
    full_path = json_dir + "/" + filename

    with open(full_path, "r") as json_data:
        data = json.load(json_data)

    return(data)

In [12]:
def save_folder(list_of_paths:list, log_filename:str) -> None:
    if os.path.exists(log_filename):
        os.remove(log_filename)
    
    with open(log_filename, "a") as log_data:
        for folder in list_of_paths:
            log_data.write(f"{folder}\n")

## Video to frames (stage 4.3)
Based on the desired fps, pick certain frames

In [13]:
""" local vars"""

thermal_fps = {
    "lower_bound": 5,
    "upper_bound": 10
}

In [14]:
""" load JSON files """

metadata_thermal = load_json(json_dir, "/thermal_png_complete.json")

In [15]:
""" log files to store exported frames """

thermal_logfile_prefix = log_dir + "/exported/thermal-exported"